# Data Collection from the Web using APIs and web scraping techniques
## CSSM530: Automated Text Processing for Social Sciences
### Table of Contents:

1. [Twitter API](#twitter-api)
    - [Authentication](#authentication)
    - [Search Tweets: Collect Tweets using keyword and/or operators](#search-tweets)
    - [Get Tweet Counts](#get-tweet-counts)
    - [Get User(s) (a.k.a User Lookup)](#get-users)
    - [Collect User's Tweets (Timeline) & Likes](#collect-users-tweets)
    - [Pagination](#pagination)
    
    
2. [Web Scraping](#web-scraping)
    - [Web Scraping with Requests & Beautiful Soup](#requests-bs4)
    - [Interacting with Websites using Selenium](#selenium)

## 1- Twitter API <a class="anchor" id="twitter-api"></a>

- Documentations:
    - [Twitter API Documentation](https://developer.twitter.com/en/docs/twitter-api)
    - [Tweepy Documentation](https://docs.tweepy.org/)
- Guide & Courses:
    - [Getting access to the Twitter API](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api)
    - [A comprehensive guide for using the Twitter API v2 with Tweepy in Python](https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9) (by Suhem Parack from TwitterDev)
    - [Getting started with the Twitter API v2 for academic research](https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research) (Official Course by TwitterDev)

In [1]:
import tweepy
import pandas as pd

### Authentication <a class="anchor" id="authentication"></a>

- Don't share your keys and tokens publicly.
- Save them in a file (e.g., .txt or .json) on your computer and read from there.

In [2]:
with open("my_api_key.txt") as read_file:
    bearer_token = read_file.read()

In [3]:
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

### Search Tweets: Collect Tweets using keyword and/or operators <a class="anchor" id="search-tweets"></a>

- Documentations:
    - **Search Recent Tweets** (search tweets from last 7 days)
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.search_recent_tweets)
    - **Search All Tweets** (full-archive search, available in **Academic Research Product Track**)
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.search_all_tweets)

- You need to specify a **search query**, **start date**, and **end date**. If you don't specify start date and end date, by default, you will get most recent tweets.
    - [How to write search queries](https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md)
    - [Building queries for Search Tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)
    - Twitter uses the following date format: **YYYY-MM-DDTHH:mm:ssZ** (ISO 8601/RFC 3339)
        - For example, 2023-03-23T16:50:00Z
        - You can write a function that converts dates in this format, e.g.,
        
        ```python
        def date_converter(date):
        """
        Convert python datetime object to ISO 8601/RFC 3339 format
        """
            return date.strftime("%Y-%m-%dT%H:%M:%SZ")
        ```
- You can get a more detailed response by specifying expansions and fields.
    - [Expansions and Fields (Tweepy)](https://docs.tweepy.org/en/stable/expansions_and_fields.html)
    - [Expansions (Twitter API)](https://developer.twitter.com/en/docs/twitter-api/expansions)
    - [Fields (Twitter API)](https://developer.twitter.com/en/docs/twitter-api/fields)
- By default, you will get 10 tweets. You can increase this number up to 100 tweets in a single request. To get more tweets, you need to use Paginator (see below).

#### Search Recent Tweets

In [4]:
# Simplest way to collect tweets, with a single keyword and default parameters
query = "seçim"

response = client.search_recent_tweets(query=query)

In [5]:
### Response
#response

- Response object includes the following fields:
    - **data**: tweets
    - **includes**: any additional fields you requested, i.e., tweet, user, media, poll & place fields 
    - **errors**
    - **meta**: newest_id, oldest_id, result_count, next_token (these allow you to paginate or collect tweets before/after a certain tweet ID)

In [6]:
### Tweets
#display(pd.DataFrame([tweet.data for tweet in response.data]))

In [7]:
# Let's specify start_time, end_time, and add a tweet field
query = "seçim"
start_time = "2023-03-20T00:00:00Z"
end_time = "2023-03-21T00:00:00Z"

response = client.search_recent_tweets(query=query,
                                       start_time=start_time,
                                       end_time=end_time,
                                       tweet_fields=["created_at"])

In [8]:
### Response
#response

In [9]:
### Tweets
#display(pd.DataFrame([tweet.data for tweet in response.data]))

#### Search All Tweets

In [10]:
query = "deprem -is:retweet"
start_time = "2023-02-06T00:00:00Z"
end_time = "2023-03-06T00:00:00Z"

response = client.search_all_tweets(query=query,
                                    start_time=start_time,
                                    end_time=end_time,
                                    expansions=["author_id", "referenced_tweets.id", "edit_history_tweet_ids",
                                                "in_reply_to_user_id", "attachments.media_keys", "attachments.poll_ids",
                                                "geo.place_id", "entities.mentions.username", "referenced_tweets.id.author_id"],
                                    tweet_fields=["id", "text", "attachments", "author_id", "context_annotations",
                                                  "conversation_id", "created_at", "entities", "geo",
                                                  "in_reply_to_user_id", "lang", "possibly_sensitive",
                                                  "public_metrics", "referenced_tweets",
                                                  "reply_settings", "source", "withheld"],
                                    user_fields=["id", "name", "username", "created_at", "description", "entities",
                                                 "location", "pinned_tweet_id", "profile_image_url", "protected",
                                                 "public_metrics", "url", "verified", "withheld"],
                                    media_fields=["media_key", "type", "url", "duration_ms", "height",
                                                  "non_public_metrics", "organic_metrics", "preview_image_url",
                                                  "promoted_metrics", "public_metrics", "width", "alt_text", "variants"],
                                    place_fields=["full_name", "id", "contained_within", "country", "country_code",
                                                  "geo", "name", "place_type"],
                                    poll_fields=["id", "options", "duration_minutes", "end_datetime", "voting_status"],
                                    max_results=100)

In [11]:
### Response
#response

In [12]:
### Tweets
#display(pd.DataFrame([tweet.data for tweet in response.data]))

In [13]:
### Users
#display(pd.DataFrame([user.data for user in response.includes["users"]]))

In [14]:
### Tweets (including referenced tweets)
#display(pd.DataFrame([tweet.data for tweet in response.includes["tweets"]]))

In [15]:
### Media
#display(pd.DataFrame([tweet.data for tweet in response.includes["media"]]))

In [16]:
### Places
#display(pd.DataFrame([place.data for place in response.includes["places"]]))

In [17]:
### Errors
#pd.DataFrame(response.errors)

## Get Tweet Counts <a class="anchor" id="get-tweet-counts"></a>
- Documentations:
    - **Recent Tweet Counts** (get tweet counts from last 7 days)
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/counts/api-reference/get-tweets-counts-recent)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_recent_tweets_count)
    - **All Tweet Counts** (full-archive search, available in **Academic Research Product Track**)
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/counts/api-reference/get-tweets-counts-all)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_all_tweets_count)
- You need to specify a **search query**, **start date**, **end date**, and **granularity** ("minute", "hour" or "day").

In [18]:
query = "lang:tr place_country:TR"
start_time = "2023-03-21T00:00:00Z"
end_time = "2023-03-22T00:00:00Z"

response = client.get_recent_tweets_count(query=query,
                                          start_time=start_time,
                                          end_time=end_time,
                                          granularity="hour")

In [19]:
### Response
#response

In [20]:
### Tweet Counts
#pd.DataFrame(response.data)[["start", "end", "tweet_count"]]

In [21]:
query = "lang:tr place_country:TR"
start_time = "2023-02-01T00:00:00Z"
end_time = "2023-02-28T00:00:00Z"

response = client.get_all_tweets_count(query=query,
                                       start_time=start_time,
                                       end_time=end_time,
                                       granularity="day")

In [22]:
### Tweet Counts
#pd.DataFrame(response.data)[["start", "end", "tweet_count"]]

### Get User(s) (a.k.a User Lookup) <a class="anchor" id="get-users"></a>

- Documentations:
    - **Get User**
        - [Twitter API, with user ID](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-id)
        - [Twitter API, with username](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-by-username-username)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_user)
    - **Get Users** (multiple accounts, up to 100 in one request)
        - [Twitter API, with user ID](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users)
        - [Twitter API, with username](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-by)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_users)

#### Get User

In [23]:
### Get user with username
response = client.get_user(username="TwitterDev")

In [24]:
#response.data.data

In [25]:
### Get user with user ID
response = client.get_user(id=2244994945)

In [26]:
#response.data.data

In [27]:
### Include user fields
response = client.get_user(username="TwitterDev",
                           user_fields=["id", "name", "username", "created_at", "description", "entities",
                                        "location", "pinned_tweet_id", "profile_image_url", "protected",
                                        "public_metrics", "url", "verified", "withheld"])

In [28]:
#response.data.data

#### Get Users

In [29]:
### Get multiple users
response = client.get_users(usernames=["TwitterDev", "Twitter"],
                            user_fields=["id", "name", "username", "created_at", "description", "entities",
                                        "location", "pinned_tweet_id", "profile_image_url", "protected",
                                        "public_metrics", "url", "verified", "withheld"])

In [30]:
#pd.DataFrame([user.data for user in response.data])

### Collect User's Tweets (Timeline) & Likes <a class="anchor" id="collect-users-tweets"></a>

- Documentations:
    - **Get User's Tweets (Timeline)**
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_users_tweets)
    - **Get User's Liked Tweets (Favorites)**
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/likes/api-reference/get-users-id-liked_tweets)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_liked_tweets)
- You can use expansions and fields to get more detailed tweet & user objects.

#### Get User's Tweets (Timeline)

In [31]:
response = client.get_users_tweets(id=2244994945,
                                   expansions="author_id",
                                   max_results=100)

In [32]:
#pd.DataFrame([tweet.data for tweet in response.data])

#### Get User's Liked Tweets (Favorites)

In [33]:
response = client.get_liked_tweets(id=2244994945,
                                   expansions="author_id",
                                   max_results=100)

In [34]:
#pd.DataFrame([tweet.data for tweet in response.data])

### Collect User's Followers & Following

- Documentations:
    - **Get User's Followers**
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_users_tweets)
    - **Get User's Following**
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/tweets/likes/api-reference/get-users-id-liked_tweets)
        - [Tweepy](https://docs.tweepy.org/en/stable/client.html#tweepy.Client.get_liked_tweets)
- You can use fields to get more detailed user objects.

In [35]:
response = client.get_users_followers(id=2244994945)

In [36]:
### Followers
#pd.DataFrame([user.data for user in response.data])

In [37]:
response = client.get_users_following(id=2244994945)

In [38]:
### Following
#pd.DataFrame([user.data for user in response.data])

### Pagination <a class="anchor" id="pagination"></a>

- Documentations:
    - **Pagination**
        - [Twitter API](https://developer.twitter.com/en/docs/twitter-api/pagination)
        - [Tweepy](https://docs.tweepy.org/en/stable/v2_pagination.html)

In [39]:
tweets = []
for tweet in tweepy.Paginator(client.search_recent_tweets, "seçim",
                              max_results=100, limit=20).flatten(limit=1000):
    tweets.append(tweet)

In [40]:
len(tweets)

1000

In [41]:
#display(pd.DataFrame(tweets))

## 2- Web Scraping <a class="anchor" id="web-scraping"></a>

- Documentations:
    - [Requests](https://requests.readthedocs.io/en/latest/)
    - [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/)
    - [Selenium](https://selenium-python.readthedocs.io/)
- Tutorials:
    - [Social ComQuant Workshop #10: Automatic Data Collection (GitHub Repo)](https://github.com/strohne/autocol)
    - [Web Scraping in Python (DataCamp)](https://app.datacamp.com/learn/courses/web-scraping-with-python)

### Web Scraping with Requests & Beautiful Soup <a class="anchor" id="requests-bs4"></a>

In [42]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [43]:
url = "https://www.tbmm.gov.tr/Tutanaklar/TutanakMetinleri"
r = requests.get(url)
print(r.status_code)

200


#### [Status Codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes):

- 1xx informational response – the request was received, continuing process
- 2xx successful – the request was successfully received, understood, and accepted
- 3xx redirection – further action needs to be taken in order to complete the request
- 4xx client error – the request contains bad syntax or cannot be fulfilled
- 5xx server error – the server failed to fulfill an apparently valid request

In [44]:
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [45]:
#print(soup.prettify())

In [46]:
table = soup.find("div", attrs={"class":"blog-content tbmm-div-list"})

In [47]:
#print(table.prettify())

In [48]:
list_groups = table.find_all("ul")

In [49]:
#list_group = list_groups[0]
#print(list_group.prettify())

In [50]:
urls = []
texts = []

for list_group in list_groups:
    urls.extend([item.find("a").get("href") for item in list_group.find_all("li")])
    texts.extend([item.find("a").text.strip() for item in list_group.find_all("li")])

In [51]:
tutanak_df = pd.concat([pd.Series(urls, name="url"), pd.Series(texts, name="description")], axis=1)

In [52]:
tutanak_df["url"] = tutanak_df["url"].apply(lambda x: "https://www.tbmm.gov.tr" + x)

In [53]:
tutanak_df["donem"] = tutanak_df["url"].apply(lambda x: x.split("?")[1].split("=")[1].split("&")[0])

In [54]:
tutanak_df.head(3)

,url,description,donem
0,https://www.tbmm.gov.tr/Tutanaklar/DoneminTuta...,6. Yasama Yılı Genel Kurul Tutanakları (1 Ekim...,27
1,https://www.tbmm.gov.tr/Tutanaklar/DoneminTuta...,5. Yasama Yılı Genel Kurul Tutanakları (1 Ekim...,27
2,https://www.tbmm.gov.tr/Tutanaklar/DoneminTuta...,4. Yasama Yılı Genel Kurul Tutanakları (1 Ekim...,27


In [55]:
def collect_tutanak_htmls(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.find("table")
    birlesim_urls = ["https://www.tbmm.gov.tr" + item.find("td").find("a").get("href") for item in table.find_all("tr")]
    
    descriptions = []
    word_urls = []
    html_urls = []
    for url in birlesim_urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        description = soup.find("div", attrs={"class": "col-8"}).text.strip()
        word_url = soup.find("a", attrs={"title":"Word Dosyası"}).get("href")
        html_url = soup.find("a", attrs={"title":"Html Dosyası"}).get("href")
        
        descriptions.append(description)
        word_urls.append(word_url)
        html_urls.append(html_url)
    return pd.concat([pd.Series(descriptions, name="description"),
                      pd.Series(word_urls, name="word_url"),
                      pd.Series(html_urls, name="html_url")], axis=1)

In [56]:
### Run the collect_tutanak_htmls function over a single url in tutanak_df
df = collect_tutanak_htmls(tutanak_df.loc[0, "url"])
display(df)

,description,word_url,html_url
0,27. DÖNEM 6. YASAMA YILI 78. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
1,27. DÖNEM 6. YASAMA YILI 77. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
2,27. DÖNEM 6. YASAMA YILI 76. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
3,27. DÖNEM 6. YASAMA YILI 75. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
4,27. DÖNEM 6. YASAMA YILI 74. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
...,...,...,...
73,27. DÖNEM 6. YASAMA YILI 5. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
74,27. DÖNEM 6. YASAMA YILI 4. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
75,27. DÖNEM 6. YASAMA YILI 3. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...
76,27. DÖNEM 6. YASAMA YILI 2. BİRLEŞİM,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...,https://cdn.tbmm.gov.tr/TbmmWeb/Tutanak2023012...


In [57]:
### Run the collect_tutanak_htmls function over all urls in tutanak_df
### Warning: This will take some time!
#
#tutanak_html_urls = pd.DataFrame()
#
#for i, url in enumerate(tutanak_df["url"]):
#    print(f"{i+1}/{len(tutanak_df)}- Collecting {url}")
#    df = collect_tutanak_htmls(url)
#    tutanak_html_urls = pd.concat([tutanak_html_urls, df]).reset_index(drop=True)
#print("Completed!")

### Interacting with Websites using Selenium <a class="anchor" id="selenium"></a>

In [58]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager # webdriver_manager allows you to easily install latest webdriver of different browsers

In [59]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()

[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST driver version for 111.0.5563
[WDM] - Driver [C:\Users\melihcanyardi\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


In [60]:
url = "https://www.tbmm.gov.tr/milletvekili-arama/form"
driver.get(url)

In [61]:
input_ad = driver.find_element(By.XPATH, """//*[@id="Adi"]""")
#input_ad.send_keys("bla bla")
#input_ad.clear()

In [62]:
select_il = Select(driver.find_element(By.XPATH, """//*[@id="IlId"]"""))
select_il.select_by_visible_text("İstanbul")

In [63]:
select_donem = Select(driver.find_element(By.XPATH, """//*[@id="DonemId"]"""))
select_donem.select_by_visible_text("Tüm Dönemler")

In [64]:
select_parti = Select(driver.find_element(By.XPATH, """//*[@id="PartiId"]"""))
select_parti.select_by_visible_text("AK Parti")

In [65]:
sorgula_button = driver.find_element(By.XPATH, """/html/body/div[1]/div/div/div[1]/div[2]/form/div[7]/div/button""")
sorgula_button.click()

In [66]:
### Scroll down
driver.find_element(By.TAG_NAME, "body").send_keys(Keys.CONTROL + Keys.END)

In [67]:
### Scroll down until the end
while True:
    try:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.CONTROL + Keys.END)
        driver.find_element(By.XPATH, """//*[@id="load-btn"]""")
    except NoSuchElementException:
        break

In [68]:
html_doc = driver.page_source
soup = BeautifulSoup(html_doc, "html.parser")
milletvekilleri = soup.find("div", attrs={"class":"blog-content tbmm-div-list"}).find_all("div", attrs={"class": "row profile-list"})

In [69]:
len(milletvekilleri)

257

In [70]:
isim_soyisim = [' '.join(milletvekili.find("div", attrs={"class":"col-8 col-md-9"}).find("strong").text.strip().split()[:-1]) for milletvekili in milletvekilleri]
gorev_listesi = [milletvekili.find("span", attrs={"class":"gorev_listesi"}).text.strip() for milletvekili in milletvekilleri]
adresler = [milletvekili.find("table").find_all("tr")[0].find_all("td")[2].text.strip() for milletvekili in milletvekilleri]
telefonlar = [milletvekili.find("table").find_all("tr")[1].find_all("td")[2].text.strip() for milletvekili in milletvekilleri]
fakslar = [milletvekili.find("table").find_all("tr")[2].find_all("td")[2].text.strip() for milletvekili in milletvekilleri]
profile_images = [milletvekili.find("div", attrs={"class":"col-4 col-md-3"}).find("img").get("src") for milletvekili in milletvekilleri]

In [71]:
milletvekilleri_df = pd.concat([pd.Series(isim_soyisim, name="isim_soyisim"),
                                pd.Series(gorev_listesi, name="gorev_listesi"),
                                pd.Series(adresler, name="adres"),
                                pd.Series(telefonlar, name="telefon"),
                                pd.Series(fakslar, name="faks"),
                                pd.Series(profile_images, name="profil_resmi")], axis=1)
milletvekilleri_df.head()

,isim_soyisim,gorev_listesi,adres,telefon,faks,profil_resmi
0,Abdullah GÜLER,"(10 / 3200, 3361, 3362, 3364, 3365) Esas Numar...",Halkla İlişkiler Binası 2 .Kat 4 Nolu Banko Od...,+90 (312) 420 57 00 +90 (312) 420 57 01,+90 (312) 420 21 51,https://www5.tbmm.gov.tr/develop/owa/display_i...
1,Ahmet Berat ÇONKAR,Dışişleri Komisyonu Başkanvekili ...,Halkla İlişkiler Binası 5 .Kat 4 Nolu Banko Od...,+90 (312) 420 54 15 +90 (312) 420 54 16,+90 (312) 420 20 43,https://www5.tbmm.gov.tr/develop/owa/display_i...
2,Ahmet Hamdi ÇAMLI,Dışişleri Komisyonu Üyesi,Halkla İlişkiler Binası 4 .Kat 3 Nolu Banko Od...,+90 (312) 420 60 26 +90 (312) 420 60 27,+90 (312) 420 25 03,https://www5.tbmm.gov.tr/develop/owa/display_i...
3,Ahmet Mücahit ARINÇ,"Milli Eğitim, Kültür, Gençlik Ve Spor Komisyon...",Halkla İlişkiler Binası Z .Kat 2 Nolu Banko Od...,+90 (312) 420 59 98 +90 (312) 420 59 99,+90 (312) 420 22 86,https://www5.tbmm.gov.tr/develop/owa/display_i...
4,Akif Çağatay KILIÇ,Dışişleri Komisyonu Başkanı,Halkla İlişkiler Binası 4 .Kat 1 Nolu Banko Od...,+90 (312) 420 53 33 +90 (312) 420 53 36,+90 (312) 420 21 13,https://www5.tbmm.gov.tr/develop/owa/display_i...


In [72]:
driver.close()